In [65]:
import numpy as np
import pandas as pd
import pymysql

In [66]:
#User can insert their comments and ratings into our database, the avgrating in Player_Bio will also update
def user_comments(PlayerName, UserName, ratings, Comments):
    #insert new entry in User_ratings
    conn = pymysql.connect(
    port=int(3306),
    user="root",
    passwd= "zzh970507",
    database = "NBA_DB")
    #Get correct ratingID
    my_cursor = conn.cursor()
    my_cursor.execute("SELECT MAX(ratingID) FROM User_ratings")
    my_result = my_cursor.fetchall()
    new_ID = int(my_result[0][0]) + 1
    
    my_cursor = conn.cursor()
    sqlFormula = "INSERT INTO User_ratings (ratings, PlayerName, UserName, UserComments, ratingID) VALUES (%s, %s, %s, %s, %s)"
    records = (ratings, PlayerName, UserName, Comments, new_ID)
    my_cursor.execute(sqlFormula, records)
    conn.commit()
    
    #update in Player_Bio
    myplayer = PlayerName
    formula_1 = "SELECT Avg(ratings) as AvgRating FROM User_ratings WHERE PlayerName = %s"
    my_cursor.execute(formula_1, myplayer)
    my_result = my_cursor.fetchall()
    new_avgRatings = float(my_result[0][0])
    my_cursor = conn.cursor()
    formula_2 = "UPDATE Player_Bio SET AvgRating = %s WHERE PlayerName = %s"
    my_cursor.execute(formula_2, (new_avgRatings, myplayer))
    conn.commit()
    conn.close()
    print("Successfully commented, your commentID is %d." % new_ID)

#query Player_Bio table by PlayerName
def return_playerBio(PlayerName):
    conn = pymysql.connect(
    port=int(3306),
    user="root",
    passwd= "zzh970507",
    database = "NBA_DB")
    my_cursor = conn.cursor()
    PlayerName = "LeBron James"
    formula_1 = "SELECT * FROM Player_Bio WHERE PlayerName = '%s'"
    sqlformula = formula_1 % PlayerName
    df = pd.read_sql(sqlformula, conn)
    print(df.to_string())
    conn.close()

#delete user rating and comments, then update average rating
def delete_ratings(deleteID):
    conn = pymysql.connect(
    port=int(3306),
    user="root",
    passwd= "zzh970507",
    database = "NBA_DB")
    
    #get playerName before deleting
    my_cursor = conn.cursor()
    formula_1 = "SELECT PlayerName FROM User_ratings WHERE ratingID = %s"
    my_cursor.execute(formula_1, deleteID)
    my_result = my_cursor.fetchall()
    Player_name = my_result[0][0]
    
    #delete record
    my_cursor = conn.cursor()
    sqlformula = "DELETE FROM User_ratings WHERE ratingID = %s"
    my_cursor.execute(sqlformula, deleteID)
    
    #update avgrating of the player in Player_Bio
    myplayer = Player_name
    formula_1 = "SELECT Avg(ratings) as AvgRating FROM User_ratings WHERE PlayerName = %s"
    my_cursor.execute(formula_1, myplayer)
    my_result = my_cursor.fetchall()
    new_avgRatings = float(my_result[0][0])
    my_cursor = conn.cursor()
    formula_2 = "UPDATE Player_Bio SET AvgRating = %s WHERE PlayerName = %s"
    my_cursor.execute(formula_2, (new_avgRatings, myplayer))
    conn.commit()
    print("Successfully deleted, the average rating of %s has been adjusted." % myplayer)
    conn.close()
    
    

In [84]:
# query player stats by playerName and Month
def query_stats_by_month(PlayerName, Month):
    conn = pymysql.connect(
    port=int(3306),
    user="root",
    passwd= "zzh970507",
    database = "NBA_DB")
    
    Months = {}
    Months["Jan"] = "01"
    Months["Feb"] = "02"
    Months["Mar"] = "03"
    Months["Apr"] = "04"
    Months["May"] = "05"
    Months["Jun"] = "06"
    Months["Jul"] = "07"
    Months["Aug"] = "08"
    Months["Sep"] = "09"
    Months["Oct"] = "10"
    Months["Nov"] = "11"
    Months["Dec"] = "12"

    my_cursor = conn.cursor()
    my_month = Months[Month]
    formula_1 = "SELECT PlayerName, Avg(Points) as Points, Avg(Rebounds) as Rebounds, Avg(Assists) as Assists, Avg(Steals) as Steals, count(*) as Game_Played FROM Game_Stats WHERE PlayerName = '%s' and CAST(Date AS CHAR) LIKE '____%s__'"
    sqlformula = formula_1 % (PlayerName, my_month)
    df = pd.read_sql(sqlformula, conn)
    print(df.to_string())
    conn.close()

def query_stats_by_opponent(PlayerName, Opponent):
    conn = pymysql.connect(
    port=int(3306),
    user="root",
    passwd= "zzh970507",
    database = "NBA_DB")

    my_cursor = conn.cursor()
    my_month = Months[Month]
    formula_1 = "SELECT PlayerName, OpponentTeam, Avg(Points) as Points, Avg(Rebounds) as Rebounds, Avg(Assists) as Assists, Avg(Steals) as Steals, count(*) as Game_Played FROM Game_Stats WHERE PlayerName = '%s' and OpponentTeam = '%s'"
    sqlformula = formula_1 % (PlayerName, Opponent)
    df = pd.read_sql(sqlformula, conn)
    print(df.to_string())
    conn.close()
    
def query_all_results_groupby_opponent(PlayerName):
    conn = pymysql.connect(
    port=int(3306),
    user="root",
    passwd= "zzh970507",
    database = "NBA_DB")

    my_cursor = conn.cursor()
    formula_1 = "SELECT PlayerName, OpponentTeam, Avg(Points) as Points, Avg(Rebounds) as Rebounds, Avg(Assists) as Assists, Avg(Steals) as Steals, count(*) as Game_Played FROM Game_Stats WHERE PlayerName = '%s' Group By OpponentTeam"
    sqlformula = formula_1 % (PlayerName)
    df = pd.read_sql(sqlformula, conn)
    print(df.to_string())
    conn.close()
    

In [59]:
return_playerBio("LeBron James")

     PlayerName TeamName  AvgPoints  AvgRebounds  AvgAssists  AvgSteals  AvgRating
0  LeBron James      LAL       27.4          8.5         8.3        1.3        4.5


In [10]:
user_comments("LeBron James", "Xiao Ming", 5, "I dont watch NBA but I know LeBron")

Successfully Commented, your commentID is 3.


In [11]:
delete_ratings(3)

Successfully deleted, the average rating of LeBron James has been adjusted.


In [85]:
query_stats_by_month("LeBron James", "Feb")

     PlayerName  Points  Rebounds  Assists  Steals  Game_Played
0  LeBron James  25.625     9.625   10.625   2.125            8


In [86]:
query_stats_by_opponent("LeBron James", "POR")

     PlayerName OpponentTeam   Points  Rebounds  Assists  Steals  Game_Played
0  LeBron James          POR  32.6667       9.0   7.3333  0.6667            3


In [87]:
query_all_results_groupby_opponent("James Harden")

      PlayerName OpponentTeam   Points  Rebounds  Assists  Steals  Game_Played
0   James Harden          OKC  35.2500    6.5000   4.0000  1.5000            4
1   James Harden          PHX  38.3333   10.0000   8.6667  3.6667            3
2   James Harden          NYK  43.5000   12.0000   6.0000  3.0000            2
3   James Harden          LAC  31.0000    5.5000  10.5000  1.0000            2
4   James Harden          SAC  39.0000    6.0000   7.2500  1.5000            4
5   James Harden          DEN  30.5000    5.5000  10.0000  1.7500            4
6   James Harden          MIL  32.5000   10.5000   6.5000  1.0000            2
7   James Harden          NOP  31.0000    8.0000   6.5000  2.7500            4
8   James Harden          SAS  37.0000    6.0000   6.0000  1.7500            4
9   James Harden          MEM  47.2500    9.5000   8.2500  2.0000            4
10  James Harden          ATL  29.5000    6.5000   7.0000  1.0000            2
11  James Harden          MIN  30.3333    3.6667   8